<a href="https://colab.research.google.com/github/Mathtzt/data-science/blob/main/apr_baseado_instancias/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo e entendimento do problema

Neste notebook será analisado o dataset do kaggle [Cardiovascular Disease dataset](https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset) a respeito de um problema de classificação, a fim de descobrir possíveis pessoas com tendências a sofrer de problemas cardíacos. Para tal, essa análise utilizará processos e técnicas próprias para uma abordagem de aprendizado de máquina baseada em instâncias.

**Descrição dos dados**

Existem 3 tipos de variáveis:

* Objective: informação factual;
* Examination: resultados de exames médicos;
* Subjective: informação dada pelo paciente.

**Features:**

1. Age | Objective Feature | age | int (days)
2. Height | Objective Feature | height | int (cm)
3. Weight | Objective Feature | weight | float (kg)
4. Gender | Objective Feature | gender | categorical code
5. Systolic blood pressure | Examination Feature | ap_hi | int
6. Diastolic blood pressure | Examination Feature | ap_lo | int
7. Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal
8. Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal
9. Smoking | Subjective Feature | smoke | binary
10. Alcohol intake | Subjective Feature | alco | binary
11. Physical activity | Subjective Feature | active | binary
12. Presence or absence of cardiovascular disease | Target Variable | cardio | binary

## Bibliotecas

In [21]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.graph_objs as go

from google.colab import files

## Importando Dados

**Uploading kaggle.json**

In [2]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**Acessando dados do Kaggle**

In [3]:
%%capture
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

**Downloading**

In [4]:
%%capture
!kaggle datasets download sulianova/cardiovascular-disease-dataset

**Unzip**

In [5]:
%%capture
!unzip cardiovascular-disease-dataset.zip

## Overview dos dados

**Lendo os dados**

In [9]:
dataframe = pd.read_csv('cardio_train.csv', sep = ';')

In [10]:
dataframe.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


**Informações básicas**

In [11]:
print(f'Quantidade de amostras = {dataframe.shape[0]}')
print(f'Quantidade de colunas|variáveis = {dataframe.shape[1]}')

Quantidade de amostras = 70000
Quantidade de colunas|variáveis = 13


In [19]:
print(f'Tipo das variáveis: \n')
dataframe.dtypes

Tipo das variáveis: 



id               int64
age              int64
gender           int64
height           int64
weight         float64
ap_hi            int64
ap_lo            int64
cholesterol      int64
gluc             int64
smoke            int64
alco             int64
active           int64
cardio           int64
dtype: object

**Medidas descritivas**

In [14]:
dataframe.describe().round(2)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.00,70000.0,70000.0
mean,49972.42,19468.87,1.35,164.36,74.21,128.82,96.63,1.37,1.23,0.09,0.05,0.8,0.5
std,28851.30,2467.25,0.48,8.21,14.40,154.01,188.47,0.68,0.57,0.28,0.23,0.4,0.5
min,0.00,10798.00,1.00,55.00,10.00,-150.00,-70.00,1.00,1.00,0.00,0.00,0.0,0.0
25%,25006.75,17664.00,1.00,159.00,65.00,120.00,80.00,1.00,1.00,0.00,0.00,1.0,0.0
50%,50001.50,19703.00,1.00,165.00,72.00,120.00,80.00,1.00,1.00,0.00,0.00,1.0,0.0
75%,74889.25,21327.00,2.00,170.00,82.00,140.00,90.00,2.00,1.00,0.00,0.00,1.0,1.0
max,99999.00,23713.00,2.00,250.00,200.00,16020.00,11000.00,3.00,3.00,1.00,1.00,1.0,1.0


**Análise de valores faltantes**

In [20]:
dataframe.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

## Análise Exploratória

### Univariada

**Idade**

Como a idade é fornecida em dias, foi necessário converter para anos, a fim de facilitar a análise e entendimento. Tendo em vista que normalmente só contabilizamos o ano depois da data de nascimento, foi realizado o truncamento dos valores, removendo as casas decimais.

Além disso, anos bisextos não serão considerados.

In [48]:
dataframe['age_years'] = dataframe['age'] / 365
## convertendo para inteiro|truncamento
dataframe['age_years'] = dataframe['age_years'].astype(int)

In [86]:
temp_ = dataframe.value_counts(subset = 'age_years').reset_index(name = 'qtd')

fig = go.Figure(data=[
      go.Bar(x = temp_['age_years'], y = temp_['qtd'], marker_color='lightsalmon')
])

fig.update_layout(
    title='Pessoas por idade',
    title_x=0.5,
    titlefont_size = 18,
    xaxis_tickfont_size=14,
    xaxis=dict(
      title='Idade',
      titlefont_size=16,
      tickfont_size=14
    ),
    yaxis=dict(
        title='Quantidade',
        titlefont_size=16,
        showticklabels=False,
        showgrid = False
    ),
    legend=dict(
        x=0,
        y=0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    template='simple_white'
)
fig.update_traces(text=temp_['qtd'], texttemplate='%{text:.3s}', textposition='outside')
fig.show()

Como observado, existem 4 amostras abaixo de 39 anos que estão destoando da distribuição. Apesar de não serem valores muito discrepantes, são registros raros no conjunto de dados e, por isso, serão removidos e analisados posteriormente.